In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
import itertools
from IPython.display import clear_output
import json
import os
import gc

from pathlib import Path, PurePath
import pandas as pd
import os

In [44]:
method = "PSO"
error = "Sammon"
metric_type = "method"

In [45]:
for path in Path("Results").glob(f"{method}_{error}_1/{metric_type}/*.json"):
    with open(path, "r") as f:
        initial_json = json.load(f)
    
    json_file_name = str(path).split('\\')[-1]
    
    new_path = f"Results/{method}_Final/{error}/{metric_type}/{json_file_name}"
    
    new_json = {}
    
    for key in initial_json:
        feat_idxs = initial_json[key]['feat_idx']
        
        feat_idxs = [x for x in feat_idxs if x <= 20]
        
        if len(feat_idxs) <= 20 and len(feat_idxs) >= 2:
            new_json[str(len(feat_idxs))] = initial_json[key]
            new_json[str(len(feat_idxs))]['feat_idx'] = feat_idxs
    with open(new_path, "w") as f:
        json.dump(new_json, f)

In [46]:
for path in Path("Results").glob(f"{method}_{error}*Extra*/{metric_type}/*.json"):
    with open(path, "r") as f:
        initial_json = json.load(f)
    
    json_file_name = str(path).split('\\')[-1]
    
    new_path = f"Results/{method}_Final/{error}/{metric_type}/{json_file_name}"
    if not os.path.exists(new_path):
        with open(new_path, "w") as f:
            json.dump(initial_json, f)
    else:
        with open(new_path, "r") as f:
            new_json = json.load(f)
        
        for key in initial_json:
            new_json[key] = initial_json[key]
                
        with open(new_path, "w") as f:
            json.dump(new_json, f)
        #print(initial_json)

In [139]:
repo_list_non_zero_metrics = set(pd.read_csv("GA_Sammon_class.csv")['Repository']).intersection(
set(pd.read_csv("PSO_Sammon_class.csv")['Repository'])).intersection(
set(pd.read_csv("PSO_Sammon_method.csv")['Repository'])).intersection(
set(pd.read_csv("GA_Sammon_method.csv")['Repository']))

repo_list_non_zero_metrics = [f"https://github.com/{x}" for x in repo_list_non_zero_metrics]

In [142]:
with open("repo_list_updated.txt", "w") as f:
    f.write('\n'.join(repo_list_non_zero_metrics))

In [168]:
method = "PSO"
error = "Sammon"
metric_type = "method"

In [169]:
df = pd.DataFrame(columns=list(range(2, 21)))
df.index.name = "Repository"

index_intersection = set(pd.read_csv("GA_Sammon_class.csv")['Repository']).intersection(
                     set(pd.read_csv("PSO_Sammon_class.csv")['Repository'])).intersection(
                     set(pd.read_csv("PSO_Sammon_method.csv")['Repository'])).intersection(
                     set(pd.read_csv("GA_Sammon_method.csv")['Repository']))


for path in Path("Results").glob(f"{method}_Final/{error}/{metric_type}/*.json"):
    with open(path, "r") as f:
        initial_json = json.load(f)
    
    new_row = [None for i in range(19)]
    
    for key in initial_json:
        #print(key)
        new_row[int(key)-2] = initial_json[key]['error_log'][-1]
    
    repo_name = str(path).split('\\')[-1][:-5].replace('_', '/')
    new_row = pd.Series(new_row, name=repo_name, index=df.columns)
    df = df.append(new_row)

df = df.loc[index_intersection]
df.replace([np.inf, -np.inf], None, inplace=True)
df = df.fillna(df.mean())

df.to_csv(f"{method}_{error}_{metric_type}_error.csv")
df.describe().loc[['mean', 'min', 'max']].reset_index().to_csv(f"{method}_{error}_{metric_type}_error_graph.csv", index=False)

In [170]:
df.describe().loc[['mean', 'min', 'max']].reset_index()

,index,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,mean,0.033576,0.014405,0.005915,0.003215,0.001607,0.000940,0.000449,0.000223,0.000125,0.000070,3.395342e-05,1.953567e-05,1.474668e-05,5.313008e-06,2.184295e-06,5.045380e-07,1.768919e-07,6.527381e-09,8.908117e-10
1,min,0.003302,0.001055,0.000523,0.000139,0.000043,0.000031,0.000017,0.000007,0.000004,0.000001,2.360529e-07,4.039125e-08,1.381103e-09,4.661040e-18,2.283069e-32,7.945484e-33,9.783446e-33,6.624088e-33,2.263145e-33
2,max,0.153661,0.100466,0.041191,0.024886,0.016567,0.011685,0.008277,0.006150,0.004458,0.002932,1.621067e-03,1.080416e-03,1.068542e-03,3.095304e-04,1.527196e-04,2.126809e-05,1.795633e-06,2.204371e-07,5.016890e-08


In [171]:
df

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
saltstack/salt,0.023005,0.010515,0.005238,0.001601,0.001019,0.000597,0.000145,0.000066,0.000036,0.000348,0.000002,7.660646e-07,9.123010e-09,6.579479e-13,5.614129e-13,4.084231e-13,8.810841e-14,5.742890e-14,1.357825e-13
pytorch/audio,0.075985,0.021375,0.012137,0.007723,0.002822,0.001909,0.000962,0.000116,0.000048,0.000018,0.000003,1.193397e-06,1.193397e-06,2.074768e-12,4.439979e-06,9.592116e-14,2.570763e-13,1.883079e-08,2.705401e-13
jina-ai/jina,0.034921,0.017192,0.006767,0.003743,0.002250,0.000962,0.000361,0.000223,0.000271,0.000007,0.000003,3.225104e-06,9.802519e-07,1.806702e-14,2.283069e-32,1.119471e-17,1.805583e-14,9.088219e-09,1.805583e-14
powerline/powerline,0.033863,0.020099,0.005682,0.002066,0.001590,0.000505,0.000237,0.000094,0.000291,0.000019,0.000003,1.538938e-06,1.468881e-06,1.886504e-14,1.468875e-06,2.605225e-17,2.605225e-17,3.032971e-32,4.370417e-32
XX-net/XX-Net,0.009211,0.005894,0.001808,0.001002,0.001607,0.000566,0.000105,0.000043,0.000011,0.000008,0.000002,8.131844e-07,5.546409e-09,3.879751e-13,2.284964e-13,1.981557e-13,2.357387e-13,3.292777e-14,1.778130e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
merenlab/anvio,0.010177,0.005708,0.002695,0.001044,0.000725,0.000263,0.000054,0.000022,0.000009,0.000019,0.000004,1.500527e-07,1.500527e-07,5.601915e-06,2.012180e-15,1.402886e-06,9.969254e-16,4.483481e-09,4.022999e-15
Rapptz/discord.py,0.027631,0.015506,0.010068,0.006430,0.002601,0.001569,0.000543,0.000225,0.000018,0.000005,0.000002,9.049816e-07,1.483507e-06,1.947352e-08,3.620791e-13,7.232700e-13,1.588937e-13,8.406243e-13,5.809176e-13
gradio-app/gradio,0.117865,0.025687,0.006197,0.002997,0.000568,0.000242,0.000234,0.000155,0.000050,0.000003,0.000001,1.315144e-06,3.103669e-06,1.192761e-13,4.908697e-13,7.557788e-14,7.722122e-14,1.090761e-08,1.959657e-14


In [173]:
index_intersection = set(pd.read_csv("GA_Sammon_class.csv")['Repository']).intersection(
set(pd.read_csv("PSO_Sammon_class.csv")['Repository'])).intersection(
set(pd.read_csv("PSO_Sammon_method.csv")['Repository'])).intersection(
set(pd.read_csv("GA_Sammon_method.csv")['Repository']))

In [174]:
pd.read_csv("GA_Sammon_class.csv").set_index("Repository").loc[index_intersection].to_csv("GA_Sammon_class.csv")
pd.read_csv("PSO_Sammon_class.csv").set_index("Repository").loc[index_intersection].to_csv("PSO_Sammon_class.csv")
pd.read_csv("GA_Sammon_method.csv").set_index("Repository").loc[index_intersection].to_csv("GA_Sammon_method.csv")
pd.read_csv("PSO_Sammon_method.csv").set_index("Repository").loc[index_intersection].to_csv("PSO_Sammon_method.csv")